In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import StrOutputParser
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [2]:
# Let's set-up our tool retrieval
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.vectorstores import FAISS
from utils import map_openbb_collection_to_langchain_tools

fundamental_openbb_tools = map_openbb_collection_to_langchain_tools("/equity/fundamental")

docs = [
    Document(page_content=t.description, metadata={"index": i})
    for i, t in enumerate(fundamental_openbb_tools)
]

vector_store = FAISS.from_documents(docs, OpenAIEmbeddings())

In [3]:
retriever = vector_store.as_retriever() # <- by default returns 4

def get_tools(query):
    docs = retriever.get_relevant_documents(query)
    return [fundamental_openbb_tools[d.metadata["index"]] for d in docs]

# Quick test
fetched_tools = get_tools("income statement")
len(fetched_tools)

4

In [4]:
# Let's wrap this in a tool
from langchain.tools import StructuredTool, Tool
fetch_tools = [
    Tool(
        name="fetch_tools",
        description="Useful for fetching a list of tools to use when answering queries.",
        func=get_tools,
    )
]

In [8]:
# Let's create a chain that can fetch tools
# We'll need a prompt, output parser, and model
from langchain.tools import format_tool_to_openai_function
from langchain.agents import AgentExecutor, initialize_agent, AgentType
from langchain.agents.format_scratchpad import format_to_openai_function_messages

system_message = """You are a helpful assistant that knows how to use tools.
When answering a query, you will always first fetch a list of tools using the `fetch_tools` function.
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_message),
        ("human", "{input}"),
    ]
)

llm = ChatOpenAI(model="gpt-3.5-turbo").bind(functions=[format_tool_to_openai_function(t) for t in fetch_tools])

runnable_agent = (
    {
        "input": lambda x: x["input"],
    }
    | prompt
    | llm
    | OpenAIFunctionsAgentOutputParser()
)

input = "What is the income statement of TSLA?"
result = runnable_agent.invoke({"input": input})

# Parse the output
if result.tool == "fetch_tools":
    fetched_tools = get_tools(result.tool_input)
    print([t.name for t in fetched_tools])
else:
    raise ValueError("Agent chose not to fetch anything.")

# Now we need to create a new chain
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

llm = ChatOpenAI(model="gpt-4-1106-preview").bind(functions=[format_tool_to_openai_function(t) for t in fetched_tools])

# Let's use ReAct
agent_executor = initialize_agent(tools=fetched_tools, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, return_intermediate_steps=True)

# runnable_agent = (
#     {
#         "input": lambda x: x["input"],
#         "agent_scratchpad": lambda x: format_to_openai_function_messages(x["intermediate_steps"])
#     }
#     | prompt
#     | llm
#     | OpenAIFunctionsAgentOutputParser()
# )

# agent_executor = AgentExecutor.from_agent_and_tools(tools=fetched_tools, agent=runnable_agent, verbose=True, return_intermediate_steps=True)

result = agent_executor.invoke({"input": input})


['income', 'income_growth', 'cash', 'balance']


> Entering new AgentExecutor chain...

Invoking: `income` with `{'symbol': 'TSLA'}`


[FMPIncomeStatementData(symbol=TSLA, date=2022-12-31, period=FY, cik=0001318605, revenue=81462000000.0, cost_of_revenue=60609000000.0, gross_profit=20853000000.0, cost_and_expenses=67630000000.0, gross_profit_ratio=0.2559843854, research_and_development_expenses=3075000000.0, general_and_administrative_expenses=0.0, selling_and_marketing_expenses=0.0, selling_general_and_administrative_expenses=3946000000.0, other_expenses=-43000000.0, operating_expenses=7021000000.0, depreciation_and_amortization=4177000000.0, ebit=None, ebitda=17833000000.0, ebitda_ratio=0.2189118853, operating_income=13656000000.0, operating_income_ratio=0.167636444, interest_income=297000000.0, interest_expense=191000000.0, total_other_income_expenses_net=63000000.0, income_before_tax=13719000000.0, income_before_tax_ratio=0.1684098107, income_tax_expense=1132000000.0, net_income=12

In [9]:
print(result['output'])

Here's the most recent annual income statement for Tesla, Inc. (TSLA) for the year ending December 31, 2022:

- Revenue: $81,462,000,000
- Cost of Revenue: $60,609,000,000
- Gross Profit: $20,853,000,000
- Selling, General and Administrative Expenses: $3,946,000,000
- Research and Development Expenses: $3,075,000,000
- Operating Expenses: $7,021,000,000
- Depreciation and Amortization: $4,177,000,000
- Operating Income: $13,656,000,000
- Interest Income: $297,000,000
- Interest Expense: $191,000,000
- Total Other Income/Expenses Net: $63,000,000
- Income Before Tax: $13,719,000,000
- Income Tax Expense: $1,132,000,000
- Net Income: $12,583,000,000
- Earnings Per Share (EPS): $4.02
- Diluted EPS: $3.62
- Weighted Average Shares Outstanding: 3,130,000,000
- Diluted Weighted Average Shares Outstanding: 3,475,000,000

The gross profit ratio for the year was approximately 25.60%, the EBITDA was $17,833,000,000 with an EBITDA ratio of around 21.89%, and the net income ratio was approximately